In [1]:
# -*- coding: utf-8 -*-
"""
Created on Wed Apr 19 13:55:28 2023

@author: Saba
"""

'''
To change at each run with different models:
    the name of the file in commands: 
        result.to_csv
    the name of ckpt in:
        callbacks      
        
This is Lidar_Transformed plus Vision data to be trained with CNN to make a multimodal network.
Image (Vision) is transformed from RGB to gray.
'''
import gc
import os
import numpy as np
import pandas as pd
from keras.layers import Conv1D, Conv2D, BatchNormalization, MaxPool1D, MaxPool2D, GlobalMaxPool1D, GlobalMaxPool2D
from keras.layers import TimeDistributed, GRU, Dense, Dropout
from tensorflow.keras.utils import to_categorical
import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow import keras
from sklearn.metrics import precision_score, recall_score
from keras.models import load_model
from keras.models import Sequential
from keras.layers import concatenate
from keras.layers import average
from keras.models import Model
from keras.models import Sequential
from sklearn.model_selection import KFold
import matplotlib.lines as mlines
import utm
from collections import Counter
import random
import cv2
import seaborn as sns
import math
import pickle 

np.random.seed(42)
tf.random.set_seed(42)  # Set TensorFlow seed as well
random.seed = 42
     
#%% Main                
#Score function
def compute_acc(y_pred, y_true, top_k=[1,3,5]):
    """ Computes top-k accuracy given prediction and ground truth labels."""
    n_top_k = len(top_k)
    total_hits = np.zeros(n_top_k)
    
    n_test_samples = len(y_true)
    if len(y_pred) != n_test_samples:
        raise Exception('Number of predicted beams does not match number of labels.')
    
    # For each test sample, count times where true beam is in k top guesses
    for samp_idx in range(len(y_true)):
        for k_idx in range(n_top_k):
            hit = np.any(y_pred[samp_idx,:top_k[k_idx]] == y_true[samp_idx, -1])
            total_hits[k_idx] += 1 if hit else 0
    
    # Average the number of correct guesses (over the total samples)
    return np.round(total_hits / len(y_true), 4)

def save_pred_to_csv(sample_index, y_pred, top_k=[1,2,3], target_csv='beam_pred.csv'):
    """ 
    Saves the predicted beam results to a csv file. 
    Expects y_pred: n_samples x N_BEAMS, and saves the top_k columns only. 
    """
    
    cols = [f'top-{i} beam' for i in top_k]
    df = pd.DataFrame(data=y_pred[:, np.array(top_k)-1], columns=cols)
    df.index.name = 'index'
    df['sample_index'] = sample_index
    df.to_csv(target_csv)

def compute_DBA_score(y_pred, y_true, max_k=3, delta=5):
    """ 
    The top-k MBD (Minimum Beam Distance) as the minimum distance
    of any beam in the top-k set of predicted beams to the ground truth beam. 
    
    Then we take the average across all samples.
    
    Then we average that number over all the considered Ks.
    """
    n_samples = y_pred.shape[0]
    #n_beams = y_pred.shape[-1] 
    
    yk = np.zeros(max_k)
    for k in range(max_k):
        acc_avg_min_beam_dist = 0
        idxs_up_to_k = np.arange(k+1)
        for i in range(n_samples):
            aux1 = np.abs(y_pred[i, idxs_up_to_k] - y_true[i]) / delta
            # Compute min between beam diff and 1
            aux2 = np.min(np.stack((aux1, np.zeros_like(aux1)+1), axis=0), axis=0)
            acc_avg_min_beam_dist += np.min(aux2)
            
        yk[k] = 1 - acc_avg_min_beam_dist / n_samples
    
    return np.mean(yk)

#%% Power factor
def compute_powerfactor(y_pred, pwrs_array, k=3):
    '''
    Calculate the maximum power factor for top-1 to top-k predictions.
    
    Args:
    y_pred (numpy array): Sorted predictions (n_samples, 64), indices of beams sorted by probability.
    pwrs_array (numpy array): Power values for beams (n_samples, 64).
    k (int): The top-k predictions to consider.
    
    Returns:
    numpy array: Array of average max PFs from top-1 to top-k.
    '''
    max_Pr = np.max(pwrs_array, axis=1)  # Maximum power across all beams for each sample
    PF_max_k = np.zeros(k)  # Array to store the average of maximum PFs for each top-k
    #PF_max_k_stds = np.zeros(k)  # Array to store the standard deviation of maximum PFs for each top-k
    
    for i in range(1, k+1):
        max_PF = np.zeros(pwrs_array.shape[0])  # Array to hold the max PF for each sample for current top-i
        for j in range(pwrs_array.shape[0]):  # Iterate over each sample
            # Calculate PFs for the top-i predictions and find the maximum
            top_k_PFs = pwrs_array[j, y_pred[j, :i]] / max_Pr[j]
            max_PF[j] = np.max(top_k_PFs)  # Maximum PF for this sample among top-i
        PF_max_k[i-1] = np.round(np.mean(max_PF), 2)  # Average of maximum PFs across all samples for top-i
        #PF_max_k_stds[i-1] = np.round(np.std(max_PF), 2)  # Standard deviation of maximum PFs for top-i

    return PF_max_k #, PF_max_k_stds

def calculate_top_beams(predictions, truths):
    correct_top1_count = 0
    correct_top3_count = 0
    total_count = len(predictions)  # Assuming predictions and truths are lists of numpy arrays
    
    for pred, true in zip(predictions, truths):
        # Find the index of the highest value in the predicted array
        top_pred_index = np.argmax(pred)
        
        # Find the indices of the top 3 highest values in the true array
        top_true_indices = np.argsort(true)[-3:]
        
        # Check if the top predicted index is among the top 3 true indices for top-1 accuracy
        if top_pred_index in top_true_indices:
            correct_top1_count += 1
        
        # Find the indices of the top 3 highest values in the predicted array
        top_pred_indices = np.argsort(pred)[-3:]
        
        # Check if there is any intersection between the top 3 predicted indices and the top 3 true indices for top-3 accuracy
        if set(top_pred_indices) & set(top_true_indices):
            correct_top3_count += 1
    
    # Calculate the percentage of correct predictions for top-1 and top-3 accuracies
    top1_accuracy = (correct_top1_count / total_count)
    top3_accuracy = (correct_top3_count / total_count) 
    top1_accuracy  = round(top1_accuracy, 2)
    top3_accuracy  = round(top3_accuracy, 2)
    return [top1_accuracy,top3_accuracy]

2024-08-25 01:47:32.964058: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-08-25 01:47:33.186721: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-08-25 01:47:34.632286: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/sa457043/miniconda3/envs/tf_jupyter/lib/:/home/sa457043/lib/python3.10/site-packa

In [2]:
'''
#%% GPU optimization
#
gpus = tf.config.list_physical_devices('GPU')
if gpus:
  try:
    # Currently, memory growth needs to be the same across GPUs
    for gpu in gpus:
      tf.config.experimental.set_memory_growth(gpu, True)
    logical_gpus = tf.config.list_logical_devices('GPU')
    print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
  except RuntimeError as e:
    # Memory growth must be set before GPUs have been initialized
    print(e)
        
#
os.environ["TF_GPU_ALLOCATOR"]="cuda_malloc_async" #to allow automatic assignment of operations to different GPUs to prevent OOM issue
'''   

# Set environment variables to disable GPU usage and use CPU instead
os.environ['CUDA_VISIBLE_DEVICES'] = '-1'  # This line disables GPU

In [3]:
#%% some global params
model_name = "LateFusion_VG_remake_v1_multipleRuns"

In [4]:
#%%Load data
def add_noise(data, noise_level):
    noisy_data = data + np.random.normal(scale=noise_level, size=data.shape)
    return noisy_data

'''
def add_gps_noise(GPS, noise_level):
    GPS_noisy = np.zeros(GPS.shape)
    GPS_noisy[:,:,0] = GPS[:,:,0] + np.random.normal(scale=noise_level, size=GPS[:,:,0].shape)
    GPS_noisy[:,:,1] = GPS[:,:,1] + np.random.normal(scale=noise_level, size=GPS[:,:,1].shape)
    return GPS_noisy
'''

df_train =  pd.read_csv('./ml_challenge_dev_multi_modal_v2.csv')
index = df_train['unit1_beam'].values

imagex = 150
imagey = 150

import cv2
def rescale(data): #to rescale from 11143x5x210x360 to 11143,5,210,225
    resized_data= np.zeros((data.shape[0],5,imagex,imagey))
    for i in range(data.shape[0]):
        for j in range(data.shape[1]):
            resized_data[i, j] = cv2.resize(data[i, j], (imagey, imagex), interpolation=cv2.INTER_NEAREST)
    return resized_data
    
data = np.load('vision_gray_11143x5x210x360.npz')
vision = rescale(data['vision'])
vision = add_noise(vision, 0.001)

'''
data = np.load('GPS_11143x5x210x360.npz')
GPS = rescale(data['GPS'])
GPS = add_noise(GPS, 0.05)
'''
GPS = np.load('GPS_11143x5x2.npz')['GPS']
print(GPS.shape)
gc.collect()


(11143, 5, 2)


22

In [5]:
gc.collect()

classes = to_categorical(df_train['unit1_beam'].values - 1, num_classes = 64, dtype ="int32")

In [6]:
#%% Models
"""Vision Model"""
def build_convnet_V(shape=(imagex,imagey)):
    momentum = .9
    
    model = keras.Sequential()
    model.add(Conv1D(4, 7, input_shape=shape,padding='same', activation='relu'))
    model.add(BatchNormalization(momentum=momentum))
    model.add(Conv1D(4, (7), padding='same', activation='relu'))
    model.add(BatchNormalization(momentum=momentum))
    model.add(MaxPool1D(pool_size=2))
    model.add(MaxPool1D(pool_size=2))

    model.add(Conv1D(16, 3, input_shape=shape,padding='same', activation='relu'))
    model.add(BatchNormalization(momentum=momentum))
    model.add(Conv1D(16, (3), padding='same', activation='relu'))
    model.add(BatchNormalization(momentum=momentum))
    model.add(MaxPool1D(pool_size=2))
    
    model.add(Conv1D(16, 3, padding='same', activation='relu'))
    model.add(BatchNormalization(momentum=momentum))
    model.add(Conv1D(16, (3), padding='same', activation='relu'))
    model.add(BatchNormalization(momentum=momentum))
    model.add(MaxPool1D(pool_size=3))

    #model.add(Conv1D(128, 3, padding='same', activation='relu'))
    #model.add(BatchNormalization(momentum=momentum))
    model.add(Conv1D(128, (3), padding='same', activation='relu'))
    model.add(BatchNormalization(momentum=momentum))
    model.add(MaxPool1D(pool_size=3))
    
    #model.add(Conv1D(256, 3, padding='same', activation='relu'))
    #model.add(BatchNormalization(momentum=momentum))
    model.add(Conv1D(256, (3), padding='same', activation='relu'))
    model.add(BatchNormalization(momentum=momentum))
    
    model.add(GlobalMaxPool1D())
    return model

def GRU_model_V(input_shape=(5,imagex,imagey), nbout=64):
    # Create our convnet with (112, 112, 3) input shape
    convnet = build_convnet_V(input_shape[1:])
    
    # then create our final model
    model = keras.Sequential()
    model.add(TimeDistributed(convnet, input_shape=input_shape))
    # here, you can also use GRU or LSTM
    model.add(GRU(64,  input_shape=input_shape))
    # and finally, we make a decision network
    model.add(Dense(64, activation='relu'))
    model.add(Dropout(.5))
    #model.add(Dense(64, activation='relu'))

    model.add(Dense(nbout, activation='softmax'))
    return model

"""GPS Model"""

def GNU_model_G(input_shape=(5, 2), nbout=64):
    model = Sequential()
    model.add(GRU(64, input_shape=input_shape))
    model.add(Dropout(0.8))

    model.add(Dense(nbout, activation='softmax'))
    return model

In [7]:
def build_model():
    V_model = GRU_model_V()
    G_model = GNU_model_G()
    V_weight,G_weight = 1, 1
    merged = concatenate([V_model.output * V_weight,
                        G_model.output * G_weight])
    z = Dense(300, activation='relu')(merged)
    z = Dense(64, activation='softmax')(z)
    model = Model(inputs=[V_model.input, G_model.input], outputs=z)
    model.compile(optimizer=keras.optimizers.Adam(0.001), loss='categorical_crossentropy', metrics=['acc'])
    print(model.summary())
    return model

In [8]:
seeds = [42, 123, 456, 789, 101112]

results = []

for seed in seeds:
    if seed == seeds[-1]:  # Check if the current seed is the last one in the list
        print(f"\nRunning model with seed {seed}")

        X_train_v, X_test_v, y_train, y_test, train_indices, test_indices = train_test_split(
            vision, classes, np.arange(len(vision)), test_size=0.1, stratify=classes, random_state=seed)
        X_train_g = GPS[train_indices]
        X_test_g = GPS[test_indices]


        #%% Scenarios and LoS NLoS analysis of train and test samples
        df_train = pd.read_csv('./ml_challenge_dev_multi_modal_v3_all_scenarios_with_LoS_status.csv')

        # Find LoS and NLoS samples in the train and test indices
        train_los_indices = df_train[df_train['LoS_status'] == 'LoS'].index.intersection(train_indices)
        train_nlos_indices = df_train[df_train['LoS_status'] == 'NLoS'].index.intersection(train_indices)
        test_los_indices = df_train[df_train['LoS_status'] == 'LoS'].index.intersection(test_indices)
        test_nlos_indices = df_train[df_train['LoS_status'] == 'NLoS'].index.intersection(test_indices)

        # Verify that all train and test samples are classified
        assert len(train_los_indices) + len(train_nlos_indices) == len(train_indices), "Mismatch in total train samples"
        assert len(test_los_indices) + len(test_nlos_indices) == len(test_indices), "Mismatch in total test samples"

        # Identify scenarios
        scenario_path = df_train['unit2_loc_1']

        # Function to count LoS and NLoS samples for a scenario
        def count_scenario_samples(scenario_keyword, los_indices, nlos_indices):
            scenario_indices = df_train[scenario_path.str.contains(scenario_keyword)].index
            los_count = len(np.intersect1d(los_indices, scenario_indices))
            nlos_count = len(np.intersect1d(nlos_indices, scenario_indices))
            return los_count, nlos_count

        # Count LoS and NLoS samples for each scenario in train and test sets
        scenario_los_nlos_counts_train = {}
        scenario_los_nlos_counts_test = {}
        for scenario_keyword in ['scenario32', 'scenario33', 'scenario34']:
            los_count_train, nlos_count_train = count_scenario_samples(scenario_keyword, train_los_indices, train_nlos_indices)
            los_count_test, nlos_count_test = count_scenario_samples(scenario_keyword, test_los_indices, test_nlos_indices)
            scenario_los_nlos_counts_train[scenario_keyword] = {'LoS': los_count_train, 'NLoS': nlos_count_train}
            scenario_los_nlos_counts_test[scenario_keyword] = {'LoS': los_count_test, 'NLoS': nlos_count_test}

        # Print the counts for each scenario in train and test sets
        print("Train Data Scenario Counts:")
        for scenario, counts in scenario_los_nlos_counts_train.items():
            print(f"{scenario} - LoS samples count: {counts['LoS']}, NLoS samples count: {counts['NLoS']}")

        print("\nTest Data Scenario Counts:")
        for scenario, counts in scenario_los_nlos_counts_test.items():
            print(f"{scenario} - LoS samples count: {counts['LoS']}, NLoS samples count: {counts['NLoS']}")

        gc.collect()

        #Run the training
        EPOCHS=300
        BS = 30

        model = build_model()

        callbacks = [
            keras.callbacks.EarlyStopping(monitor='val_loss', patience=20, mode='min', verbose=1),
            keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=8, mode='min', verbose=1),
            keras.callbacks.ModelCheckpoint(
                f'chkp/{model_name}_seed_{seed}.hdf5', 
                monitor='val_acc',  # Monitor validation accuracy
                save_best_only  = True,
                mode='max',  # Maximize the monitored quantity
                verbose=1),
        ]

        history = model.fit(
            x=[X_train_v, X_train_g],
            y=y_train,
            validation_split = 0.2, #0
            #validation_data=(X_val, y_val),
            verbose=1,
            #verbose='auto', #auto
            epochs=EPOCHS,
            batch_size =BS,
            callbacks=callbacks
        )

        gc.collect()


Running model with seed 101112
Train Data Scenario Counts:
scenario32 - LoS samples count: 2692, NLoS samples count: 104
scenario33 - LoS samples count: 3330, NLoS samples count: 120
scenario34 - LoS samples count: 3684, NLoS samples count: 98

Test Data Scenario Counts:
scenario32 - LoS samples count: 302, NLoS samples count: 17
scenario33 - LoS samples count: 371, NLoS samples count: 16
scenario34 - LoS samples count: 398, NLoS samples count: 11


2024-08-22 20:17:18.658147: E tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:267] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2024-08-22 20:17:18.658179: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:169] retrieving CUDA diagnostic information for host: edison
2024-08-22 20:17:18.658183: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:176] hostname: edison
2024-08-22 20:17:18.658316: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:200] libcuda reported version is: 560.28.3
2024-08-22 20:17:18.658337: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:204] kernel reported version is: NOT_FOUND: could not find kernel module information in driver version file contents: "NVRM version: NVIDIA UNIX Open Kernel Module for x86_64  560.28.03  Release Build  (dvs-builder@U16-A24-27-4)  Thu Jul 18 20:46:24 UTC 2024
GCC version:  gcc version 11.4.0 (Ubuntu 11.4.0-1ubuntu1~22.

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 time_distributed_input (InputL  [(None, 5, 150, 150  0          []                               
 ayer)                          )]                                                                
                                                                                                  
 time_distributed (TimeDistribu  (None, 5, 256)      113536      ['time_distributed_input[0][0]'] 
 ted)                                                                                             
                                                                                                  
 gru (GRU)                      (None, 64)           61824       ['time_distributed[0][0]']       
                                                                                              

In [8]:
#%% Test on all scens together
#%%Test on Train and Test data (top-1,2,3)

# Model evaluation and prediction
def evaluate_model(model, X_data, y_data, pwrs_array, data_type, result_list, sample_indices):
    predictions = model.predict(X_data)
    y_pred = np.argsort(predictions, axis=1)[:, ::-1]
    save_pred_to_csv(sample_indices, y_pred, top_k=[1,2,3], target_csv=f'preds_{model_name}_{data_type}.csv')
    true = np.argmax(y_data, axis=1).reshape(-1, 1)
    acc = compute_acc(y_pred, true, top_k=[1, 3, 5])
    score = compute_DBA_score(y_pred, true, max_k=3, delta=5)
    recall = recall_score(true, y_pred[:, 0], average='weighted')
    precision = precision_score(true, y_pred[:, 0], average='weighted')
    PF_mean = compute_powerfactor(y_pred, pwrs_array, k=3)
    top_beams = calculate_top_beams(predictions, pwrs_array)
   
    result_list.append({
        'type': data_type,
        'acc': acc,
        'score': round(score, 2),
        'top31_beam': top_beams[0],
        'top33_beam': top_beams[1],
        'PF1_mean': PF_mean[0],
        'PF2_mean': PF_mean[1],
        'PF3_mean': PF_mean[2],
        'recall': round(recall, 2),
        'precision': round(precision, 2)
    })
    
# Define the seeds used for training
seeds = [42, 123, 456, 789, 101112]

# Placeholder for storing results from all seeds
all_results = []

for seed in seeds:
    print(f"\nEvaluating model with seed {seed}")

    # Train-Test Split with the current seed
    X_train_v, X_test_v, y_train, y_test, train_indices, test_indices = train_test_split(
        vision, classes, np.arange(len(vision)), test_size=0.1, stratify=classes, random_state=seed)
    X_train_g = GPS[train_indices]
    X_test_g = GPS[test_indices]
    
    df_train = pd.read_csv('./ml_challenge_dev_multi_modal_v3_all_scenarios_with_LoS_status.csv')

    # Find LoS and NLoS samples in the train and test indices
    train_los_indices = df_train[df_train['LoS_status'] == 'LoS'].index.intersection(train_indices)
    train_nlos_indices = df_train[df_train['LoS_status'] == 'NLoS'].index.intersection(train_indices)
    test_los_indices = df_train[df_train['LoS_status'] == 'LoS'].index.intersection(test_indices)
    test_nlos_indices = df_train[df_train['LoS_status'] == 'NLoS'].index.intersection(test_indices)

    gc.collect()

    X_test_los_v, X_test_nlos_v = vision[test_los_indices], vision[test_nlos_indices]
    X_test_los_g, X_test_nlos_g = GPS[test_los_indices], GPS[test_nlos_indices]
    y_test_los, y_test_nlos = classes[test_los_indices], classes[test_nlos_indices]

    # Power analysis - make sure to get the correct power arrays for each seed
    N_CLASSES = 64
    pwrs_array = np.zeros((df_train.shape[0], N_CLASSES))
    for sample_idx in range(df_train.shape[0]):
        pwr_abs_path = df_train['unit1_pwr_60ghz'].values[sample_idx]
        pwrs_array[sample_idx] = np.loadtxt(pwr_abs_path)

    pwrs_array[np.isnan(pwrs_array)] = 0
    pwrs_array_train = pwrs_array[train_indices]
    pwrs_array_test_los = pwrs_array[test_los_indices]
    pwrs_array_test_nlos = pwrs_array[test_nlos_indices]

    # Load the model
    model = load_model(f'chkp/{model_name}_seed_{seed}.hdf5')

    # Initialize results list
    seed_results = []

    # Evaluate on training data
    evaluate_model(model, [X_train_v, X_train_g], y_train, pwrs_array_train, 'Train', seed_results, train_indices)

    # Evaluate on test data
    evaluate_model(model, [X_test_los_v,X_test_los_g], y_test_los, pwrs_array_test_los, 'Test_LoS', seed_results, test_los_indices)
    evaluate_model(model, [X_test_nlos_v,X_test_nlos_g], y_test_nlos, pwrs_array_test_nlos, 'Test_NLoS', seed_results, test_nlos_indices)

    # Store the results of this seed in all_results
    all_results.append(seed_results)

# Function to average results across all seeds
def average_results(all_results, metric):
    averages = {}
    types = ['Train', 'Test_LoS', 'Test_NLoS']
    for data_type in types:
        type_results = [res for seed_res in all_results for res in seed_res if res['type'] == data_type]
        avg_metric = np.mean([res[metric] for res in type_results], axis=0)
        std_metric = np.std([res[metric] for res in type_results], axis=0)
        averages[data_type] = (avg_metric, std_metric)
    return averages

# Compute averages and standard deviations for each metric
avg_accuracies = average_results(all_results, 'acc')
avg_scores = average_results(all_results, 'score')
avg_top31_beams = average_results(all_results, 'top31_beam')
avg_top33_beams = average_results(all_results, 'top33_beam')
avg_PF1_means = average_results(all_results, 'PF1_mean')
avg_PF2_means = average_results(all_results, 'PF2_mean')
avg_PF3_means = average_results(all_results, 'PF3_mean')
avg_recalls = average_results(all_results, 'recall')
avg_precisions = average_results(all_results, 'precision')

# Print the average results across all seeds
print("\nAverage Results Across All Seeds:")
for data_type in avg_accuracies:
    print(f"Type: {data_type} - Avg Accuracies: {avg_accuracies[data_type][0]} ± {avg_accuracies[data_type][1]}, "
          f"Avg Score: {avg_scores[data_type][0]} ± {avg_scores[data_type][1]}, "
          f"top31_beam: {avg_top31_beams[data_type][0]} ± {avg_top31_beams[data_type][1]}, "
          f"top33_beam: {avg_top33_beams[data_type][0]} ± {avg_top33_beams[data_type][1]}, "
          f"PF1_mean: {avg_PF1_means[data_type][0]} ± {avg_PF1_means[data_type][1]}, "
          f"PF2_mean: {avg_PF2_means[data_type][0]} ± {avg_PF2_means[data_type][1]}, "
          f"PF3_mean: {avg_PF3_means[data_type][0]} ± {avg_PF3_means[data_type][1]}, "
          f"Recall: {avg_recalls[data_type][0]} ± {avg_recalls[data_type][1]}, "
          f"Precision: {avg_precisions[data_type][0]} ± {avg_precisions[data_type][1]}")

gc.collect()


Evaluating model with seed 42


2024-08-25 01:52:45.188776: E tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:267] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2024-08-25 01:52:45.188804: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:169] retrieving CUDA diagnostic information for host: edison
2024-08-25 01:52:45.188809: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:176] hostname: edison
2024-08-25 01:52:45.188955: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:200] libcuda reported version is: 560.35.3
2024-08-25 01:52:45.188978: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:204] kernel reported version is: NOT_FOUND: could not find kernel module information in driver version file contents: "NVRM version: NVIDIA UNIX Open Kernel Module for x86_64  560.35.03  Release Build  (dvs-builder@U16-I1-N07-12-3)  Fri Aug 16 21:42:42 UTC 2024
GCC version:  gcc version 11.4.0 (Ubuntu 11.4.0-1ubuntu1~

314/314 [==============================] - 8s 24ms/step


/home/sa457043/miniconda3/envs/tf_jupyter/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


2/2 [==============================] - 0s 31ms/step


...

/home/sa457043/miniconda3/envs/tf_jupyter/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


2/2 [==============================] - 0s 15ms/step


/home/sa457043/miniconda3/envs/tf_jupyter/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/sa457043/miniconda3/envs/tf_jupyter/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/sa457043/miniconda3/envs/tf_jupyter/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))



Average Results Across All Seeds:
Type: Train - Avg Accuracies: [0.44486 0.77192 0.87752] ± [0.07338391 0.07945007 0.05571059], Avg Score: 0.8400000000000001 ± 0.04774934554525328, top31_beam: 0.748 ± 0.08953211714239753, top33_beam: 0.9199999999999999 ± 0.041952353926806046, PF1_mean: 0.9560000000000001 ± 0.018547236990991395, PF2_mean: 0.9799999999999999 ± 0.008944271909999166, PF3_mean: 0.986 ± 0.00489897948556636, Recall: 0.446 ± 0.07172168430816443, Precision: 0.392 ± 0.10684568311354464
Type: Test_LoS - Avg Accuracies: [0.39192 0.74422 0.87408] ± [0.053405   0.05664625 0.04462154], Avg Score: 0.8379999999999999 ± 0.041182520563947986, top31_beam: 0.73 ± 0.0812403840463596, top33_beam: 0.9279999999999999 ± 0.03544009029333868, PF1_mean: 0.9540000000000001 ± 0.020591260281973965, PF2_mean: 0.976 ± 0.01019803902718558, PF3_mean: 0.986 ± 0.00489897948556636, Recall: 0.39 ± 0.05215361924162119, Precision: 0.318 ± 0.07386474125047755
Type: Test_NLoS - Avg Accuracies: [0.01928 0.0481  

18667

In [9]:
# Define a function to format mean ± std
def format_mean_std(mean, std):
    return f"{mean:.2f} ± {std:.2f}"

# Initialize the final results list
formatted_results = []

# Process the results for saving
for data_type in avg_accuracies:
    acc_means, acc_stds = avg_accuracies[data_type]
    row = {
        'Type': data_type,
        'Acc': ', '.join([format_mean_std(m, s) for m, s in zip(acc_means, acc_stds)]),
        'Score': format_mean_std(*avg_scores[data_type]),
        'top31_beam': format_mean_std(*avg_top31_beams[data_type]),
        'top33_beam': format_mean_std(*avg_top33_beams[data_type]),
        'PF1_mean': format_mean_std(*avg_PF1_means[data_type]),
        'PF2_mean': format_mean_std(*avg_PF2_means[data_type]),
        'PF3_mean': format_mean_std(*avg_PF3_means[data_type]),
        'Recall': format_mean_std(*avg_recalls[data_type]),
        'Precision': format_mean_std(*avg_precisions[data_type])
    }
    formatted_results.append(row)

# Convert the list to a DataFrame
df = pd.DataFrame(formatted_results)

# Save to CSV
df.to_csv(f'results_{model_name}_AllScens.csv', index=False)

print("Results saved to CSV:")
print(df)

Results saved to CSV:
        Type                                    Acc        Score   top31_beam  \
0      Train  0.44 ± 0.07, 0.77 ± 0.08, 0.88 ± 0.06  0.84 ± 0.05  0.75 ± 0.09   
1   Test_LoS  0.39 ± 0.05, 0.74 ± 0.06, 0.87 ± 0.04  0.84 ± 0.04  0.73 ± 0.08   
2  Test_NLoS  0.02 ± 0.02, 0.05 ± 0.03, 0.09 ± 0.05  0.09 ± 0.04  0.26 ± 0.05   

    top33_beam     PF1_mean     PF2_mean     PF3_mean       Recall  \
0  0.92 ± 0.04  0.96 ± 0.02  0.98 ± 0.01  0.99 ± 0.00  0.45 ± 0.07   
1  0.93 ± 0.04  0.95 ± 0.02  0.98 ± 0.01  0.99 ± 0.00  0.39 ± 0.05   
2  0.41 ± 0.06  0.88 ± 0.02  0.91 ± 0.02  0.92 ± 0.01  0.02 ± 0.02   

     Precision  
0  0.39 ± 0.11  
1  0.32 ± 0.07  
2  0.02 ± 0.02  
